## Import libraries

In [ ]:
!pip install PyPDF2
!pip install xlsxwriter
!pip install nltk

In [ ]:
import pandas as pd
from google.colab import drive
import os
import IPython
import PyPDF2
import shutil
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

## Connect with drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

## Create folder

In [ ]:
# Define the folder path in your Google Drive
general_path= '/content/drive/My Drive/TrabajoWater/Archivos'

# Check if the folder exists, if not, create it
if not os.path.exists(general_path):
    os.makedirs(general_path)
    print("Folder was created")

## Extract text from pdf - Full paper

In [ ]:
# Function to extract text from PDF files
def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file, strict=False)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [ ]:
def extract_REFERENCES_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)

        # Initialize variables
        references_text = ""
        references_found = False

        # Loop through each page
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            page_text = page.extract_text()

            # Check if "REFERENCES" is in the page text
            if "REFERENCES" in page_text:
                references_found = True

                # Append the content starting from "REFERENCES" to the result
                references_index = page_text.find("REFERENCES")
                references_text += page_text[references_index:]

                # Append text from subsequent pages
                for next_page_num in range(page_num + 1, num_pages):
                    next_page = reader.pages[next_page_num]
                    next_page_text = next_page.extract_text()
                    references_text += next_page_text

                # No need to process further pages, break the loop
                break
        print(references_text.strip())
        return references_text.strip()

In [ ]:
# Array to store text from all files
texts = []
titles = []

# Folder with the pdf files
path_with_pdfs = os.path.join(general_path)

# Loop through each file in the folder
for filename in os.listdir(path_with_pdfs):
    file_path = os.path.join(path_with_pdfs, filename)



    try:
      # Check if the file is a PDF
      if filename.endswith('.pdf'):
          text = extract_text_from_pdf(file_path

          print(f"Title: {filename}")

          references_text = extract_REFERENCES_from_pdf(file_path)

          # Remove REFERENCES text from main text
          if references_text:
              print(references_text)
              text = text.replace(references_text, '')

          # Update the texts list with the modified text
          texts.append(text)
          titles.append(filename)
          print("="*50)
    except:
      continue

In [ ]:
df = pd.DataFrame({'FullText': texts,'Title': titles})

In [ ]:
df.head()

In [ ]:
df.to_csv(general_path, "archivos.csv", escapechar="\\")